In [227]:
import os
import re
import folium
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm.notebook import tqdm

def listdir_nohidden(path):
    def ld_nh(path):
        for f in os.listdir(path):
            if not f.startswith('.'):
                yield f
    return list (ld_nh(path))

import geopandas as gp

### run the notebook once for each election

In [228]:
data_dir = '../raw_turnout_by_precinct'
elections = [e for e in listdir_nohidden(data_dir) if '.' not in e ]
elections

['2018_november', '2016_may', '2018_may']

#### (cycle this index 0-2 through 2:)

In [245]:
e_idx = 2
election = elections[e_idx]

target_dir = f'../{election.lower()}_cleaned'

if target_dir[3:] not in os.listdir('..'):
    os.makedirs(target_dir)
print(target_dir)

../2018_may_cleaned


### identify data for both races (`init` + `runoff`)

In [246]:
e_races = listdir_nohidden(os.path.join(data_dir, election )) 


# if parties are separated
if 'DEM' in os.listdir(os.path.join(data_dir, election, e_races[0])):
    party = 'DEM'
else:
    party = ''

init_race = [c+'/'+party for c in e_races if 'Runoff' not in c][0]
runoff_race = [c+'/'+party for c in e_races if 'Runoff' in c][0]



init_race_files = listdir_nohidden(os.path.join(data_dir, election, init_race )) 
runoff_race_files = listdir_nohidden(os.path.join(data_dir, election, runoff_race )) 

try: assert init_race_files == runoff_race_files
except:
    print('Initial election data does not match final elections data')
    init_only = [i for i in init_race_files if i not in runoff_race_files]
    runoff_only = [i for i in runoff_race_files if i not in init_race_files]
    print('Missing from initial data:', runoff_only)
    print('Missing from runoffs data:', init_only)

e_races = {'INIT':
              {'name': init_race,
               'files': init_race_files,},
          
          'RUNOFF': 
              {'name': runoff_race,
               'files': runoff_race_files,},}

[race['name'] for race in e_races.values()]

['May 2018/DEM', 'June 2018 (Runoff)/DEM']

### parse data for one county in this election

In [247]:
def parse_pair_data(county):
    out = []
    for race_type, race in e_races.items(): # iterate county data for both INIT & RUNOFF

        county_file = os.path.join(data_dir, election, race['name'], county )

        data = pd.read_excel(county_file)

        #election_date = data.columns[0].split('\n')[-1].split(':')[1]

        # quick clean and specify columns
        data.loc[0] = data.T[0].fillna(method='ffill')
        data.loc[1] = data.loc[1]

        data.columns = (data.loc[1] + '_' + data.loc[0]
                       ).apply(
                            lambda x: 
                               str(x).replace('Reg #', 'N_registered'
                                    ).replace('Voted', 'N_voted'
                                    ).replace('%', 'PCT_voted'
                                    ).replace(' ', '_')
                                if str(x)!='nan' else ''                             
                            )
            
        data = data.rename(columns={k: race_type+'_'+k for k in data.columns})

        
        
        # grab index names from subsequent row & consolidate
        data.columns = list(data.loc[0].values[:2]) + list(data.columns)[2:]
        data = data[2:].reset_index(drop=True)

        out.append(data)
    
    out = pd.merge(out[0], out[1], on='PRECINCT ID', how='outer')
    
    out['County'] = county.split('.')[0]
    
    # calc participation rate change
    out = out[['County']+[c for c in out.columns if c!='County']]
    return out

out = parse_pair_data(sorted(init_race_files)[0])

### combine data for one election (init + runoff)

In [248]:
e_data = pd.DataFrame()
for county in tqdm(init_race_files):
    out = parse_pair_data(county)
    #if 'STEP' in county: break
    e_data = pd.concat([e_data, out])
e_data.reset_index(drop=True, inplace=True)
e_data.drop('PRECINCT DESCRIPTION_y', axis=1, inplace=True)
e_data.rename(columns={'PRECINCT DESCRIPTION_x': 'PRECINCT DESCRIPTION'}, inplace=True)

In [249]:
# drop duplicate columns
e_data = e_data.loc[:,~e_data.columns.duplicated()]

e_data = e_data.reset_index(drop=True)
e_data = e_data.sort_values('PRECINCT ID').sort_values('County')

# engineer sums for specific groups

In [250]:
for c in e_data.columns:
    e_data = e_data.rename(columns={c:c.replace('TOTAL_VOTERS', 'ALL_ALL')})
e_data.head()

,County,PRECINCT ID,PRECINCT DESCRIPTION,INIT_N_registered_BLACK_MALE,INIT_N_voted_BLACK_MALE,INIT_PCT_voted_BLACK_MALE,INIT_N_registered_BLACK_FEMALE,INIT_N_voted_BLACK_FEMALE,INIT_PCT_voted_BLACK_FEMALE,INIT_N_registered_BLACK_UNKNOWN,...,RUNOFF_PCT_voted_UNKNOWN_MALE,RUNOFF_N_registered_UNKNOWN_FEMALE,RUNOFF_N_voted_UNKNOWN_FEMALE,RUNOFF_PCT_voted_UNKNOWN_FEMALE,RUNOFF_N_registered_UNKNOWN,RUNOFF_N_voted_UNKNOWN,RUNOFF_PCT_voted_UNKNOWN,RUNOFF_N_registered_ALL_ALL,RUNOFF_N_voted_ALL_ALL,RUNOFF_PCT_voted_ALL_ALL
1659,APPLING,4D,4D,28,7,25.00,30,9,30,0,...,0,45,0,0,0,0,0,1420,5,0.35
1660,APPLING,5A,5A,34,3,8.82,55,5,9.09,0,...,0,44,0,0,0,0,0,1091,6,0.55
1661,APPLING,5B,5B,49,3,6.12,62,5,8.06,0,...,3.12,27,0,0,0,0,0,1078,4,0.37
1657,APPLING,3C,3C,141,15,10.64,180,31,17.22,0,...,2.08,52,1,1.92,0,0,0,1569,12,0.76
1654,APPLING,1C,1C,8,1,12.50,18,0,0,0,...,0,29,0,0,0,0,0,812,5,0.62


### create dataframe of group identity column keys 

In [251]:
ppl_stats = [c for c in e_data.columns if 'MALE' in c] # captures all people (interesting)

ppl_stat_dict = {} # compile list of columns relevant to each parent group type
for colname in ppl_stats:
    cparts = colname.split('_')
    ppl_stat_dict[colname] = {
        'sex': 'FEMALE' if 'FEMALE' in cparts[-1] else 'MALE',
        'race': cparts[-2]   } # get dummy descriptions for both categories in each column
    
ppl_cols_df = pd.DataFrame(ppl_stat_dict
    ).T.reset_index().rename(columns={'index':'col_name'})

print('Sexes:', ppl_cols_df.sex.unique())
print('Races:', ppl_cols_df.race.unique())

# create column groups for each parent person group

group_agg_map = {}

for p_idx in ppl_cols_df.index:    
    p_row = ppl_cols_df.loc[p_idx]
    
    if p_row.sex not in group_agg_map.keys():
        group_agg_map[p_row.sex] = []
    group_agg_map[p_row.sex].append(p_row.col_name)
    
    if p_row.race not in group_agg_map.keys():
        group_agg_map[p_row.race] = []
    group_agg_map[p_row.race].append(p_row.col_name)

print(len(group_agg_map), 'total identity groups defined')

# create custom super all-all category
group_agg_map['ALL'] = ppl_stats

Sexes: ['MALE' 'FEMALE']
Races: ['BLACK' 'WHITE' 'ASIA-PI' 'HISP-LT' 'NATIVE-AM' 'OTHER' 'UNKNOWN']
9 total identity groups defined


In [252]:
# function to get the sum total for each statistic in a parent identity group
def calc_group_sum(group):
    
    # id all cols for this group, drop PCT (all just calced.)
    target_cols = [c for c in group_agg_map[group] if 'pct' not in c.lower()]
    
    # group cols by stat type
    stat_cats = list(sorted(set(
        ['_'.join(c.split('_')[:-2]) for c in target_cols])))
    # the above are be universal ( [INIT/R.O.] + [REG/VOTED] )    
    
    # for the given group....
    stat_cols = {}
    for stat in stat_cats: # iterate to get sum of each statistic type
        
        stat_cols[stat] = [] # to compile relevant columns for this statistic
        for col in target_cols: 
            if stat in col: # identify target cols for THIS STAT
                stat_cols[stat].append(col) 
        
        if 'MALE' in group:
            sum_col_name = stat+'_ALL_'+group
        else:
            sum_col_name = stat+'_'+group+'_ALL' 
        
        
        e_data[sum_col_name] = e_data[ # this is where the addition occurs
                                stat_cols[stat]  # columns to add
                                     ].astype(float).sum(axis=1) # ADD ACROSS AXIS 1  
        
for group in group_agg_map.keys():
    calc_group_sum(group)

## create groupby'd version

In [253]:
for c in e_data.columns:
    try: e_data[c] = e_data[c].astype(float)
    except: pass
    
g_data = e_data.groupby('County').sum()

## calc delta & pct change stats for all groups

In [254]:
elecs = ['INIT', 'RUNOFF']
stats = ['N_voted', 'N_registered']

get_group = lambda x: '_'.join(x.split('_')[3:])

# ALL groups (original and engineered sums)
all_group_names = set([get_group(c) for c in e_data.columns[3:]])

for group in sorted(all_group_names):
    
    target_cols = [c for c in e_data.columns if group == get_group(c)]
        
    for stat in stats:
        stat_cols = [c for c in target_cols if stat in c]        
        
        delta_colname = stat_cols[0].replace('INIT', 'Delta')
        e_data[delta_colname] = e_data[stat_cols[1]].astype(float) - e_data[stat_cols[0]].astype(float)
        g_data[delta_colname] = g_data[stat_cols[1]].astype(float) - g_data[stat_cols[0]].astype(float)

        
        pct_colname = stat_cols[0].replace('INIT', 'pctChange')
        e_data[pct_colname] = e_data[delta_colname] / e_data[stat_cols[0]].astype(float)
        g_data[pct_colname] = g_data[delta_colname] / g_data[stat_cols[0]].astype(float)
        

### clean up feature names a bit more 

In [255]:
cols = list(e_data.columns)
re_map = {}
for col in cols[3:]:
    c = col.split('_')
    elec = c[0].capitalize()
    if 'Pct' in elec:
        elec = 'pctChange'
    stat = c[1].lower()+c[2].capitalize()
    race = c[3].capitalize()
    if len(c)>=5: sex = c[4].capitalize()
    else: sex = 'Unknown'        
    re_map[col] = f"{elec}_{stat}_{race}_{sex}"

e_data = e_data.rename(columns=re_map)
g_data = g_data.rename(columns=re_map)

In [256]:
# FUNCTION FOR DISPLAYING / RETURN DATA FOR A TARGET GROUP

def get_group_data(df, race, sex, elec=None):
    target_cols = []
    o_cols = []
    for col in df:
        if col.count('_')<3:
            o_cols.append(col)
            continue
        c = col.split('_')
        if not race or c[2] == race:
            if not elec or c[0] == elec:
                if not sex or c[3] == sex:
                    target_cols.append(col)
                        
    return df[ o_cols + target_cols ]

### re-calc group participation rates

In [257]:
elecs = ['Init', 'Runoff']
stats = ['nVoted', 'nRegistered']
races = sorted(set([c.split('_')[2] for c  in g_data.columns[3:]]))
sexes = sorted(set([c.split('_')[3] for c  in g_data.columns[3:]]))

for sex in sexes:
    for race in races:
        if race=='All' and sex=='Unknown': continue # doesnt exist
        
        for elec in elecs:
            group_data = get_group_data(g_data, race, sex, elec)

            for stat in stats:
                group_data[[c for c in group_data.columns if any(s in c for s in stats) ]]

            cols = group_data.columns
            if not len(cols):
                print(race, sex, elec)
                out = group_data
                

            target_fname = f"{elec}_pctVoted_{race}_{sex}"
            g_data[target_fname] = round(100*(g_data[cols[1]] / g_data[cols[0]]), 2)
            e_data[target_fname] = round(100*(e_data[cols[1]] / e_data[cols[0]]), 2)

## create state totals dataframe

In [258]:
vote_dict = {
    'Init': {},
    'Runoff': {},  }
regi_dict = {
    'Init': {},
    'Runoff': {},  }

for sex in sexes:
    for race in races:
        if race=='All' and sex=='Unknown': continue # doesnt exist
        for elec in elecs:
            group_data = get_group_data(g_data, race, sex, elec)
            
            category = '_'.join(group_data.columns[1].split('_')[2:])
            
            n_votes = group_data[group_data.columns[1]].sum()
            n_reg = group_data[group_data.columns[0]].sum()

            vote_dict[str(elec)][category] = n_votes
            regi_dict[str(elec)][category] = n_reg
            
state_totals = pd.DataFrame(vote_dict)

# quick features
state_totals = state_totals.rename(columns=
                {c:c.lower()+'Votes' for c in state_totals.columns})

# add n_registered back on
r_data = pd.DataFrame(regi_dict)
r_data = r_data.rename(columns=
                {c:c.lower()+'Registered' for c in r_data.columns})
state_totals = pd.merge(state_totals, r_data, left_index=True, right_index=True)

state_totals['initParticipation'] = round(100*(state_totals['initVotes'] / state_totals['initRegistered']), 2)
state_totals['runoffParticipation'] = round(100*(state_totals['runoffVotes'] / state_totals['runoffRegistered']), 2)

# sort columns for init then runoff
state_totals = state_totals[sorted(state_totals.columns)]



# CALCULATED FEATURES
state_totals['nVotesChange'] = state_totals['runoffVotes'] - state_totals['initVotes']


state_totals['ParticipationChange'] = round(100*(state_totals['nVotesChange'] / state_totals['initVotes']), 2)

# sort by group participation dropoff
#state_totals = state_totals.sort_values('ParticipationChange')

# OPTIONAL: ADD COMMA SEPARATORS FOR THOUSANDS (no longer numeric after this obviously)
for c in state_totals.columns:
    if 'Partic' not in c:
        state_totals[c] = state_totals.apply(lambda x:
                        "{:,}".format(int(x[c])) if str(x[c])!='nan' else '-', axis=1)
    else :
        state_totals[c] = state_totals.apply(lambda x:
                        "{}%".format(int(x[c]))  if str(x[c])!='nan' else '-', axis=1)

# CONVERT INDEX TO TWO SEPARATE COLUMNS
parse_race = lambda x: x.split('_')[0]
parse_sex = lambda x: x.split('_')[1]
state_totals['race'] = [parse_race(v) for v in state_totals.index]
state_totals['sex'] = [parse_sex(v) for v in state_totals.index]
cols = list(state_totals.columns)
state_totals = state_totals[cols[-2:]+cols[:-2]]

state_totals = state_totals.sort_values('sex')
state_totals = state_totals.sort_values('race')
state_totals = state_totals.replace('All', '(all)')

if party:
    fname = target_dir+f'/statewide_{party}_participation.csv'
    state_totals.drop('ParticipationChange', axis=1, inplace=True)
    
else:
    fname = target_dir+'/statewide_participation.csv'


state_totals.to_csv(fname, index=False) # index is now redundant
print(fname, 'saved:')
state_totals

../2018_may_cleaned/statewide_DEM_participation.csv saved:


,race,sex,initParticipation,initRegistered,initVotes,runoffParticipation,runoffRegistered,runoffVotes,nVotesChange
All_All,(all),(all),8%,"6,733,692","562,785",2%,"6,684,291","159,888","-402,897"
All_Male,(all),Male,6%,"3,108,872","212,160",1%,"3,077,890","58,263","-153,897"
All_Female,(all),Female,9%,"3,624,820","350,625",2%,"3,606,401","101,625","-249,000"
Asia-pi_All,Asia-pi,(all),4%,"137,678","6,865",1%,"137,130","2,705","-4,160"
Asia-pi_Unknown,Asia-pi,Unknown,4%,365,17,3%,365,13,-4
Asia-pi_Male,Asia-pi,Male,4%,"65,475","3,159",2%,"65,162","1,304","-1,855"
Asia-pi_Female,Asia-pi,Female,5%,"72,203","3,706",1%,"71,968","1,401","-2,305"
Black_All,Black,(all),16%,"2,037,709","338,724",4%,"2,018,655","95,291","-243,433"
Black_Unknown,Black,Unknown,5%,"2,427",141,1%,"2,383",30,-111
Black_Male,Black,Male,13%,"867,355","119,187",3%,"854,943","32,017","-87,170"


## save

In [259]:
# PRECINCT DATA
if party:
    elec_fpath = target_dir+f'/all_precincts_{party}_participation.csv'
else:
    elec_fpath = target_dir+'/all_precincts_participation.csv'
print('Saving precinct data...')
print('>', elec_fpath, '\n')
e_data.to_csv(elec_fpath, index=False)

# COUNTY DATA
if party:
    elec_fpath = target_dir+f'/all_counties_{party}_participation.csv'
else:
    elec_fpath = target_dir+'/all_counties_participation.csv'
print('Saving county data...')
print('>', elec_fpath)
g_data.to_csv(elec_fpath)

Saving precinct data...
> ../2018_may_cleaned/all_precincts_DEM_participation.csv 

Saving county data...
> ../2018_may_cleaned/all_counties_DEM_participation.csv


In [244]:
get_group_data(e_data, 'Black', 'Male')

,County,PRECINCT ID,PRECINCT DESCRIPTION,Init_nRegistered_Black_Male,Init_nVoted_Black_Male,Init_pctVoted_Black_Male,Runoff_nRegistered_Black_Male,Runoff_nVoted_Black_Male,Runoff_pctVoted_Black_Male,Delta_nVoted_Black_Male,pctChange_nVoted_Black_Male,Delta_nRegistered_Black_Male,pctChange_nRegistered_Black_Male
1631,APPLING,4C,4C,2.0,0.0,0.00,2.0,0.0,0.0,0.0,NaN,0.0,0.000000
1629,APPLING,4A,4A,4.0,0.0,0.00,4.0,0.0,0.0,0.0,NaN,0.0,0.000000
1632,APPLING,4D,4D,21.0,3.0,14.29,21.0,0.0,0.0,-3.0,-1.0,0.0,0.000000
1633,APPLING,5A,5A,34.0,2.0,5.88,35.0,0.0,0.0,-2.0,-1.0,1.0,0.029412
1634,APPLING,5B,5B,47.0,7.0,14.89,46.0,0.0,0.0,-7.0,-1.0,-1.0,-0.021277
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1319,WORTH,15,ISABELLA,54.0,4.0,7.41,57.0,0.0,0.0,-4.0,-1.0,3.0,0.055556
1327,WORTH,8,OAKFIELD,35.0,5.0,14.29,35.0,0.0,0.0,-5.0,-1.0,0.0,0.000000
1330,WORTH,99999,99999,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN
1320,WORTH,16,PINEY WOODS,45.0,10.0,22.22,45.0,0.0,0.0,-10.0,-1.0,0.0,0.000000
